In [16]:
from pathlib import Path

import optuna
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

from sklearn.model_selection import cross_validate

from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
import mlflow
from optuna.integration import MLflowCallback

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
# Set tracking URI
MODEL_REGISTRY = Path("experiments")
#Path(MODEL_REGISTRY).mkdir(exist_ok=True)  # create experiments dir
mlflow.set_tracking_uri("file:///" + str(MODEL_REGISTRY.absolute()))

In [3]:
df = pd.read_csv("../data/proccessed/globalterrordb_proccesed.csv", index_col=0)

In [5]:
df.fillna(value=-9, inplace=True)

Podział danych

In [44]:
y = df["cas_class"]
X = df.drop("cas_class", axis=1)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify=y)

In [45]:
# X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42, test_size=0.2, stratify=y)
# X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, random_state=42, test_size=0.25, stratify=y_train)

Transformacje danych

In [46]:
impute_value = -9

In [47]:
pipeline = make_pipeline(SimpleImputer(missing_values=np.nan, strategy="constant", fill_value=impute_value),
                         StandardScaler())

In [48]:
pipeline.fit_transform(X_train)

array([[-0.2035149 ,  0.60373644,  1.10873672, ..., -1.08014204,
        -0.85284415, -0.52435801],
       [-0.2035149 ,  0.95885428,  0.68576861, ...,  1.06308836,
         1.90854282,  1.78257866],
       [-0.2035149 , -0.4616171 , -0.13845736, ..., -1.08014204,
         0.2747222 ,  0.24575256],
       ...,
       [-0.2035149 , -0.81673495, -0.44779573, ..., -1.08014204,
        -0.34430395, -0.52435801],
       [-0.2035149 , -0.81673495, -0.94373996, ..., -1.08014204,
        -0.5943832 , -0.52435801],
       [-0.2035149 ,  0.95885428,  0.54991043, ..., -1.08014204,
         1.90854282, -0.52435801]])

In [49]:
# pipeline.transform(X_val)
pipeline.transform(X_test)

array([[-0.2035149 ,  1.31397213, -1.293771  , ...,  1.06308836,
        -0.96759715, -0.40496541],
       [-0.2035149 ,  0.95885428,  0.53947394, ..., -1.08014204,
         1.90854282, -0.52435801],
       [-0.2035149 ,  0.95885428,  0.54991043, ..., -1.08014204,
         1.90854282, -0.52435801],
       ...,
       [-0.2035149 ,  0.95885428,  0.54991043, ..., -1.08014204,
         1.90854282, -0.52435801],
       [-0.2035149 , -0.81673495, -0.93188165, ...,  0.84876532,
        -0.34430395,  0.52810758],
       [-0.2035149 , -2.23720633,  1.02973381, ..., -1.08014204,
        -0.7749706 , -0.51417286]])

Wyszukiwanie hiperparametrów w regresji logistycznej

In [48]:
def objective(trial):
    params = {
        "C": trial.suggest_float("C", 0.5, 20, log=True),
        "class_weight": trial.suggest_categorical("class_weight", ["balanced"]),
        "n_jobs": trial.suggest_categorical("n_jobs", [-1]),
        "max_iter": trial.suggest_categorical("max_iter", [250, 500])
    }

    model = LogisticRegression(**params)
    # model.fit(X_train, y_train)
    # preds = model.predict(X_val)
    #
    # accuracy = accuracy_score(y_val, preds)
    # precision = precision_score(y_val, preds)
    # recall = recall_score(y_val, preds)
    # f1 = f1_score(y_val, preds)
    scoring = ["accuracy", "precision", "recall", "f1"]

    preds = cross_validate(model, X_train, y_train, cv=5, n_jobs=-1, scoring=scoring)

    accuracy = np.mean(preds["test_accuracy"])
    precision = np.mean(preds["test_precision"])
    recall = np.mean(preds["test_recall"])
    f1 = np.mean(preds["test_accuracy"])

    return accuracy, precision, recall, f1


In [42]:
mlflow_callback = MLflowCallback(
    tracking_uri=mlflow.get_tracking_uri(), metric_name=["accuracy", "precision", "recall", "f1"])

C:\Users\Tuszyn\AppData\Local\Temp\ipykernel_4668\3644345758.py:1: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlflow_callback = MLflowCallback(


In [49]:
study = optuna.create_study(study_name="lr", directions=["maximize", "maximize", "maximize", "maximize"])
study.optimize(objective, n_trials=10, callbacks=[mlflow_callback])

[I 2022-05-26 17:59:38,694] A new study created in memory with name: lr
C:\Users\Tuszyn\.virtualenvs\JT_praca_magisterska\lib\site-packages\optuna\distributions.py:427: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 0.35, 1: 0.65} which is of type dict.
  warnings.warn(message)
[I 2022-05-26 17:59:47,584] Trial 0 finished with values: [0.7087058233770575, 0.8222002434570906, 0.6997992618014635, 0.7560779375240493] and parameters: {'C': 3.074532702811549, 'class_weight': 'balanced', 'n_jobs': -1, 'max_iter': 250}. 
C:\Users\Tuszyn\.virtualenvs\JT_praca_magisterska\lib\site-packages\optuna\distributions.py:427: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains {0: 0.35, 1: 0.65} which is of type dict.
  warnings.warn(message)
[I 2022-05-26 17:59:55,411] Trial 1 finished with values: [0.6945024647004763, 

Wyszukiwanie hiperparametrów drzewo decyzyjne

In [66]:
def objective(trial):
    params = {
        "max_depth": trial.suggest_int("max_depth", 10, 40),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 2, 50),
        "max_features": trial.suggest_categorical("max_features", [None]),
        "class_weight": trial.suggest_categorical("class_weight", ["balanced"]),
    }

    model = DecisionTreeClassifier(**params)
    # model.fit(X_train, y_train)
    # preds = model.predict(X_val)

    scoring = ["accuracy", "precision", "recall", "f1"]

    preds = cross_validate(model, X_train, y_train, cv=5, n_jobs=-1, scoring=scoring)

    accuracy = np.mean(preds["test_accuracy"])
    precision = np.mean(preds["test_precision"])
    recall = np.mean(preds["test_recall"])
    f1 = np.mean(preds["test_accuracy"])

    # accuracy = accuracy_score(y_val, preds)
    # precision = precision_score(y_val, preds)
    # recall = recall_score(y_val, preds)
    # f1 = f1_score(y_val, preds)

    return accuracy, precision, recall, f1

In [67]:
study = optuna.create_study(study_name="dt", directions=["maximize", "maximize", "maximize", "maximize"])
study.optimize(objective, n_trials=50, callbacks=[mlflow_callback])

[I 2022-05-26 20:18:01,426] A new study created in memory with name: dt
[I 2022-05-26 20:18:04,080] Trial 0 finished with values: [0.8217126142721753, 0.8975337826218288, 0.8169573307890354, 0.8217126142721753] and parameters: {'max_depth': 10, 'min_samples_leaf': 8, 'max_features': None, 'class_weight': 'balanced'}. 
[I 2022-05-26 20:18:06,079] Trial 1 finished with values: [0.8286613636907246, 0.904225473340154, 0.8214470115081639, 0.8286613636907246] and parameters: {'max_depth': 29, 'min_samples_leaf': 44, 'max_features': None, 'class_weight': 'balanced'}. 
[I 2022-05-26 20:18:07,805] Trial 2 finished with values: [0.8304298769419866, 0.9056939169933793, 0.8228714649324068, 0.8304298769419866] and parameters: {'max_depth': 33, 'min_samples_leaf': 33, 'max_features': None, 'class_weight': 'balanced'}. 
[I 2022-05-26 20:18:08,677] Trial 3 finished with values: [0.8306944436863107, 0.905996203381983, 0.822958018737211, 0.8306944436863107] and parameters: {'max_depth': 17, 'min_samples

Testowo

In [13]:
pipeline = make_pipeline(SimpleImputer(missing_values=np.nan, strategy="constant", fill_value=-9),
                         StandardScaler())

In [14]:
pipeline.fit_transform(X_train)
pipeline.transform(X_test)

array([[-0.20454128,  0.2512775 ,  1.11346144, ...,  0.84629142,
        -0.78830057, -0.50202743],
       [-0.20454128, -1.52336728, -2.05252351, ..., -1.08281702,
        -0.414869  , -0.52463359],
       [-0.20454128, -0.45858041,  0.01885846, ..., -1.08281702,
        -0.87121762, -0.44268628],
       ...,
       [-0.20454128,  1.67099332, -3.10661319, ..., -1.08281702,
        -1.05656162, -0.52463359],
       [-0.20454128, -1.52336728, -1.18432961, ...,  0.84629142,
        -0.24141385, -0.46585759],
       [-0.20454128, -0.45858041,  0.45137931, ..., -1.08281702,
         0.49813307, -0.44268628]])

In [37]:
def objective(trial):
    params = {
        "C": trial.suggest_float("C", 50, 200, log=True),
        "class_weight": trial.suggest_categorical("class_weight", ["balanced"]),
        "max_iter": trial.suggest_categorical("max_iter", [250])
    }

    model = LogisticRegression(**params)
    # model.fit(X_train, y_train)

    scoring = ["accuracy", "precision", "recall", "f1"]

    preds = cross_validate(model, X_train, y_train, cv=5, n_jobs=-1, scoring=scoring)
    # accuracy = accuracy_score(y_val, preds)
    # precision = precision_score(y_val, preds)
    # recall = recall_score(y_val, preds)
    # f1 = f1_score(y_val, preds)

    accuracy = np.mean(preds["test_accuracy"])
    precision = np.mean(preds["test_precision"])
    recall = np.mean(preds["test_recall"])
    f1 = np.mean(preds["test_accuracy"])

    return accuracy, precision, recall, f1

In [ ]:
mlflow_callback = MLflowCallback(
    tracking_uri=mlflow.get_tracking_uri(), metric_name=["accuracy", "precision", "recall", "f1"])

In [38]:
study = optuna.create_study(study_name="test_lr", directions=["maximize", "maximize", "maximize", "maximize"])
study.optimize(objective, n_trials=10)

[I 2022-05-26 19:02:16,922] A new study created in memory with name: test_lr
[I 2022-05-26 19:02:20,958] Trial 0 finished with values: [0.6905136617922907, 0.8106852254488347, 0.6788622916555699, 0.6905136617922907] and parameters: {'C': 108.73463114655324, 'class_weight': 'balanced', 'max_iter': 250}. 
[I 2022-05-26 19:02:25,286] Trial 1 finished with values: [0.7039551506687899, 0.8168858042826542, 0.6975117036758666, 0.7039551506687899] and parameters: {'C': 171.54036997853433, 'class_weight': 'balanced', 'max_iter': 250}. 
[I 2022-05-26 19:02:29,217] Trial 2 finished with values: [0.6968741779970103, 0.8100456699119688, 0.693221641070414, 0.6968741779970103] and parameters: {'C': 177.17063855981704, 'class_weight': 'balanced', 'max_iter': 250}. 
[I 2022-05-26 19:02:32,877] Trial 3 finished with values: [0.6945974619539199, 0.8090807907742367, 0.6891094776139498, 0.6945974619539199] and parameters: {'C': 170.4886598205659, 'class_weight': 'balanced', 'max_iter': 250}. 
[I 2022-05-26